In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import util

In [ ]:
smgp = util.SMGP('../smgp.asm')
tracks = smgp.extract_track_data()

In [ ]:
def render_slope_data(track_data):
    slope_data_loc = util.extract_track_data_part('slope data', track_data)
    slope_data = smgp.extract_loc_data(slope_data_loc)
    slope_derivative = np.cumsum(list(util.rle_decode(util.parse_slope_data(slope_data))))
    return np.cumsum(slope_derivative)

In [ ]:
fig, ax = plt.subplots(len(tracks), 1, figsize=(13,28))

i=0
for name, track_data in tracks.items():
    ax[i].plot(render_slope_data(track_data))
    ax[i].set_title(name)
    i+=1

plt.tight_layout()
plt.show()

### Curve

In [ ]:
curve_data_loc = util.extract_track_data_part('curve data', tracks['Hungary'])
curve_data = smgp.extract_loc_data(curve_data_loc)
util.parse_curve_data(curve_data)

In [ ]:
# Nonlinear transformation of curve sharpness to try and make it match up with minimap

def transform_exp(x):
    turn_factor = 3.5 # 8.5
    if x == 0:
        return 0
    elif x < 0:
        return -np.exp(-x/turn_factor)+1
    else:
        return np.exp(x/turn_factor)-1

def transform_poly(x):
    turn_factor = 16 # 4.5
    if x == 0:
        return 0
    elif x < 0:
        return -(-x/47)**turn_factor
    else:
        return (x/47)**turn_factor

def transform(rle):
    return [(length, transform_poly(v)) for length, v in rle]

accumulated_turning = np.cumsum(list(util.rle_decode(transform(util.parse_curve_data(curve_data)))))

In [ ]:
turn90 = 23*transform_poly(47)

plt.plot(
    np.cumsum(np.sin(accumulated_turning/turn90*np.pi/2)),
    np.cumsum(np.cos(accumulated_turning/turn90*np.pi/2))
)
plt.plot(0, 0, 'X')
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
plt.plot(np.exp(np.arange(48)/8.35)/np.exp(47/8.35))
plt.plot(np.linspace(0, 1, 48)**4.5)